# Automated Model Discovery for Deli Meat

Meat Model Discovery Papers:
1. "Discovery the mechanics of artificial and real meat": https://doi.org/10.1016/j.cma.2023.116236
2. "The mechanical and sensory signature of plant-based and animal meat": https://www.nature.com/articles/s41538-024-00330-6

Code by Skyler St. Pierre, Ethan Darwin, Steven Tran, Kevin Linka \\
Last edited December 2024


### 0. Load python packages

In [ ]:
!pip install tensorflow==2.12.0

In [ ]:
# make sure to only restart session after all 3 packages have finished i.e. hit "cancel" on the pop-up after the first two
!pip install matplotlib==3.2.2
!pip install numpy==1.23.5
!pip install pandas==1.5.3

Make sure to select `Runtime` > `Restart Session` as the top of the file before running the next code block!

In [ ]:
# Essentials
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import pandas as pd
import os
import copy
# ML
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras as keras
from tensorflow.keras import regularizers
from sklearn.metrics import r2_score
# Others
import pickle
import json
import statistics
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl

#!pip install openpyxl

# Check Versions
print('Numpy: ' + np.__version__)
print('Matplotlib: ' + matplotlib.__version__) # 3.2.2
print('Tensorflow: ' + tf.__version__) # 2.12.0
print('Keras: ' + keras.__version__)
print('Pandas: ' + pd.__version__) # 1.5.3

In [ ]:
!python --version
import sklearn
print(sklearn.__version__)
print(json.__version__)

In [ ]:
#Import excel file, change to match where you saved the file
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/Colab Notebooks/SURI24" # change to where you download this; must be in Google Drive

###Useful Functions


In [ ]:
def makeDIR(path):
    if not os.path.exists(path):
        os.makedirs(path)

def flatten(l):
    return [item for sublist in l for item in sublist]

def r2_score_own(Truth, Prediction):
    R2 = r2_score(Truth,Prediction)
    return max(R2,0.0)

def count_directories(path):
    try:
        all_items = os.listdir(path)

        directories = [item for item in all_items if os.path.isdir(os.path.join(path, item))]
        return 1#len(directories)+1
    except:
        return 0

### 1. Load Skin data

In [ ]:
# Material to train the CANN for
filename_prefix = 'Realpros' # 'Realchicken', 'Realturkey', 'Realham', 'Tofurkyturkey', 'Tofurkyham','Tofurkyhickory', 'Fakepros', 'Realpros'
cwd = os.getcwd()

# Data Selection
zero_data = True # If True, data starts from lambda=1, sigma=0
truncate_data = 21 # Number of data points per loading mode (strip-x, off-x, etc.)

###Import Data

In [ ]:
# Make sure you have right data directory as: pd.read_excel('data location')
# break_list = where to trim the data (or equal to data size of each biaxial mode)

if filename_prefix=='Realchicken':
    df_MechData = pd.read_excel(path + '/Input/rm_chicken_results.xlsx')
    break_list = [truncate_data,truncate_data+21,truncate_data+42,truncate_data+63,truncate_data+84]
elif filename_prefix=='Realturkey':
    df_MechData = pd.read_excel(path + '/Input/rm_turk_homo_results.xlsx')
    break_list = [truncate_data,truncate_data+21,truncate_data+42,truncate_data+63,truncate_data+84]
elif filename_prefix=='Realham':
    df_MechData = pd.read_excel(path + '/Input/rm_ham_homo_results.xlsx')
    break_list = [truncate_data,truncate_data+21,truncate_data+42,truncate_data+63,truncate_data+84]
elif filename_prefix=='Tofurkyturkey':
    df_MechData = pd.read_excel(path + '/Input/tf_turk_rake_results.xlsx')
    break_list = [truncate_data,truncate_data+21,truncate_data+42,truncate_data+63,truncate_data+84]
elif filename_prefix=='Tofurkyham':
    df_MechData = pd.read_excel(path + '/Input/tf_ham_results.xlsx')
    break_list = [truncate_data,truncate_data+21,truncate_data+42,truncate_data+63,truncate_data+84]
elif filename_prefix=='Tofurkyhickory':
    df_MechData = pd.read_excel(path + '/Input/tf_hick_results.xlsx')
    break_list = [truncate_data,truncate_data+21,truncate_data+42,truncate_data+63,truncate_data+84]
elif filename_prefix=='Realpros':
    df_MechData = pd.read_excel(path + '/Input/rm_pros_results.xlsx')
    break_list = [truncate_data,truncate_data+21,truncate_data+42,truncate_data+63,truncate_data+84]
elif filename_prefix=='Fakepros':
    df_MechData = pd.read_excel(path + '/Input/fm_pros_results.xlsx')
    break_list = [truncate_data,truncate_data+21,truncate_data+42,truncate_data+63,truncate_data+84]
elif filename_prefix=='Tofurkyturkey_clamp':
    df_MechData = pd.read_excel(path + '/Input/tf_turk_clamp_results.xlsx')
    break_list = [truncate_data,truncate_data+21,truncate_data+42,truncate_data+63,truncate_data+84]
else:
    ValueError()

# Data conversion
lambda_x =  df_MechData['lambda_x'].to_numpy()
sigma_xx =  df_MechData['sigma_xx[kPa]'].to_numpy()

lambda_y =  df_MechData['lambda_y'].to_numpy()
sigma_yy =  df_MechData['sigma_yy[kPa]'].to_numpy()

# Seperate data from each biaxial mode
st=0
all_lam_x = []
all_lam_y = []
all_Sigma_xx = []
all_Sigma_yy = []
for i, end in enumerate(break_list): # has no effect if data already starts at lambda=1, stress=0 for each mode
    all_lam_x.append(lambda_x[st:end]-(lambda_x[st]-1)*zero_data)
    all_lam_y.append(lambda_y[st:end]-(lambda_y[st]-1)*zero_data)
    all_Sigma_xx.append(sigma_xx[st:end]-sigma_xx[st]*zero_data)
    all_Sigma_yy.append(sigma_yy[st:end]-sigma_yy[st]*zero_data)
    st = end + 1

### L1 and L2 regularization with penalty weight



In [ ]:
def regularize(reg, pen):
    if reg == 'L2':
        return keras.regularizers.l2(pen)
    if reg == 'L1':
        return keras.regularizers.l1(pen)

## 2. Strain Energy Model - Invariant-based
<figure>
<center>
<img src='https://drive.google.com/uc?export=view&id=1-0eM6j7CP_jzSXM_AswZU8P7-IzEN1I5'/>
<figcaption>Isotropic CANN</figcaption></center>
</figure>



Next, we define the strain energy function for our transversely isotropic, perfectly incompressible Constitutive Artificial Neural Network with two hidden layers and 8 nodes using the invariants of the right Cauchy Green tensor. The first layer generates powers $(\circ)^1$ and $(\circ)^2$ of the network inputs,
$[I_1-3]$, $[I_2-3]$, and the second layer applies the identity, $(\circ)$ and the exponential function, $(\rm{exp}((\circ))-1)$.
The set of equations for this networks takes the following explicit form,

$$
\begin{equation}
\begin{split}
\psi(I_1,I_2) &= w_{1} w_{1}^*[I_1-3] + w_{2}[\exp(w_{2}^*[I_1-3])-1]\\
& + w_{3} w_{3}^*[I_1-3]^2  + w_{4}[\exp(w_{4}^*[I_1-3]^2)-1] \\
& + w_{5} w_{5}^*[I_2-3] + w_{6}[\exp(w_{6}^*[I_2-3])-1] \\
& + w_{7} w_{7}^*[I_2-3]^2 + w_{8}[\exp(w_{8}^*[I_2-3]^2)-1]\\
\end{split}
\end{equation}
$$

First we define the activation functions and a single Invariant block:

In [ ]:
# Initializers (look up https://keras.io/api/layers/initializers/ for more)
initializer_1 = 'glorot_normal'
initializer_zero = 'zeros'
initializer_log = 'glorot_normal'
initializer_exp = tf.keras.initializers.RandomUniform(minval=0.5, maxval=2.5) # worked off and on, starts with huge residual
initializer_exp2 = tf.keras.initializers.RandomUniform(minval=0.01, maxval=0.1)

# Self defined activation functions for exp term
def activation_Exp(x):
    return 1.0*(tf.math.exp(x) -1.0)
# Self defined activation functions for ln term
def activation_ln(x):
    return -1.0*tf.math.log(1.0 - (x))

# Define Invariant building-blocks(linear, quad, exp, exp-quad, ln)
def SingleInvNet_5(I1_ref,idi,reg,pen):
    I_1_w11 = keras.layers.Dense(1,kernel_initializer=initializer_1,kernel_constraint=keras.constraints.NonNeg(),
                                 kernel_regularizer=regularize(reg, pen),
                                 use_bias=False, activation=None,name='w'+str(1+idi)+'1')(I1_ref)
    I_1_w21 = keras.layers.Dense(1,kernel_initializer=initializer_exp,kernel_constraint=keras.constraints.NonNeg(),
                                 kernel_regularizer=regularize(reg, pen),
                                 use_bias=False, activation=activation_Exp,name='w'+str(2+idi)+'1')(I1_ref)

    I_1_w31 = keras.layers.Dense(1,kernel_initializer=initializer_1,kernel_constraint=keras.constraints.NonNeg(),
                                 kernel_regularizer=regularize(reg, pen),
                                 use_bias=False, activation=None,name='w'+str(3+idi)+'1')(tf.math.square(I1_ref))
    I_1_w41 = keras.layers.Dense(1,kernel_initializer=initializer_exp,kernel_constraint=keras.constraints.NonNeg(),
                                 kernel_regularizer=regularize(reg, pen),
                                 use_bias=False, activation=activation_Exp,name='w'+str(4+idi)+'1')(tf.math.square(I1_ref))

    collect = [I_1_w11, I_1_w21, I_1_w31, I_1_w41]
    collect_out = tf.keras.layers.concatenate(collect, axis=1)

    return collect_out

# Define Invariant building-blocks(linear, quad, exp, ln)
def SingleInvNet_4(I1_ref,idi,reg,pen):
    I_1_w11 = keras.layers.Dense(1,kernel_initializer=initializer_1,kernel_constraint=keras.constraints.NonNeg(),
                                 kernel_regularizer=regularize(reg, pen),
                                 use_bias=False, activation=None,name='w'+str(1+idi)+'1')(I1_ref)
    I_1_w21 = keras.layers.Dense(1,kernel_initializer=initializer_exp,kernel_constraint=keras.constraints.NonNeg(),
                                 kernel_regularizer=regularize(reg, pen),
                                 use_bias=False, activation=activation_Exp,name='w'+str(2+idi)+'1')(I1_ref)
    I_1_w31 = keras.layers.Dense(1,kernel_initializer=initializer_1,kernel_constraint=keras.constraints.NonNeg(),
                                 kernel_regularizer=regularize(reg, pen),
                                 use_bias=False, activation=None,name='w'+str(3+idi)+'1')(tf.math.square(I1_ref))

    collect = [I_1_w11, I_1_w21, I_1_w31]
    collect_out = tf.keras.layers.concatenate(collect, axis=1)

    return collect_out

Then we define the strain energy keras submodel as:

In [ ]:
def StrainEnergy_i5(reg,pen):

    # Inputs defined
    I1_in = tf.keras.Input(shape=(1,), name='I1')
    I2_in = tf.keras.Input(shape=(1,), name='I2')

    # Invariants reference config
    I1_ref = keras.layers.Lambda(lambda x: (x-3.0))(I1_in)
    I2_ref = keras.layers.Lambda(lambda x: (x-3.0))(I2_in)

    I1_out = SingleInvNet_5(I1_ref,0,reg,pen)
    terms = I1_out.get_shape().as_list()[1]
    I2_out = SingleInvNet_5(I2_ref,terms,reg,pen)

    ALL_I_out = tf.keras.layers.concatenate([I1_out,I2_out],axis=1)

    # second layer
    W_ANN = keras.layers.Dense(1,kernel_initializer=initializer_1,kernel_constraint=keras.constraints.NonNeg(),
                               kernel_regularizer=regularize(reg, pen),
                               use_bias=False, activation=None,name='wx2')(ALL_I_out)
    Psi_model = keras.models.Model(inputs=[I1_in, I2_in], outputs=[W_ANN], name='Psi')

    return Psi_model, terms*2

### 3. Stress Models


####  Biaxial Tension

For the case of biaxial tension, we stretch the specimen in two directions,
$F_{11} = \lambda_1 = \lambda$ and $F_{22} = \lambda_2 = \lambda$.
For an isotropic, perfectly incompressible material with
$I_3 = \lambda_1^2  \lambda_2^2  \lambda_3^2 = 1$,
the stretch orthogonal to the loading directions are identical and equal to the inverse square of the stretch,
$F_{33} = \lambda_3 = \lambda^{-2}$.
From the resulting deformation gradient,
$\mathbf{F}= {\rm{diag}} \, \{ \; \lambda, \lambda, \lambda^{-2} \,\}$,
we calculate the invariants and their derivatives,

$$
\begin{align*}
    I_{1} &= {\lambda_{1}}^2+{\lambda_{2}}^2+\frac{1}{{\lambda_{1}}^2{\lambda_{2}}^2} && \frac{\partial I_{1}}{\partial\mathbf{F}} = 2\,\text{diag}\left\{\lambda_{1},\lambda_{2},\frac{1}{\lambda_{1}\lambda_{2}}\right\}\\
    I_{2} &= {\lambda_{1}}^2{\lambda _{2}}^2+\frac{1}{{\lambda_{1}}^2}+\frac{1}{{\lambda_{2}}^2} && \frac{\partial I_{2}}{\partial\mathbf{F}} = 2\,\text{diag}\left\{ \lambda_{1}{\lambda_{2}}^{2}+\frac{1}{\lambda_{1}{\lambda_{2}}^{2}},{\lambda_{1}}^{2}\lambda_{2}+\frac{1}{{\lambda _{1}}^{2}\lambda_{2}},\frac{\lambda_{1}}{\lambda_{2}}+\frac{\lambda_{2}}{\lambda_{1}}\right\}\\
    I_{3} &= 1 && \frac{\partial I_{3}}{\partial\mathbf{F}} = 2\,\text{diag}\left\{\frac{1}{\lambda_{1}},\frac{1}{\lambda_{2}},\lambda_{1}\lambda_{2}\right\}\\
\end{align*}
$$

to evaluate the nominal biaxial stress $P_{11}$
using the general stress-stretch relationship for perfectly incompressible materials,
$ \begin{equation*}
    \mathbf{P}=\frac{\partial \psi_{1}}{\partial I_{1}}\frac{\partial I_{1}}{\partial \mathbf{F}} + \frac{\partial \psi_{2}}{\partial I_{2}}\frac{\partial I_{2}}{\partial \mathbf{F}} - p\mathbf{F}^{-t}
\end{equation*}$.
Here, $p$ denotes the hydrostatic pressure that we determine from the zero stress condition in the transverse directions, $P_{33} = 0$, as
$ \begin{equation*}
    p=\frac{2}{{\lambda_{1}}^{2}{\lambda_{2}}^{2}}\frac{\partial \psi_{1}}{\partial I_{1}}\left(\frac{1}{{\lambda_{1}}^{2}}+\frac{1}{{\lambda_{2}}^{2}}\right)
\end{equation*}.$
This results in the following explicit biaxial stress-stretch relation for perfectly incompressible, transversely isotropic materials,

$$
\begin{align*}
    P_{11}=2\left(\lambda_{1}-\frac{1}{{\lambda_{1}}^{3}{\lambda_{2}}^{2}}\right)\frac{\partial \psi_{1}}{\partial I_{1}} + 2\left(\lambda_{1}{\lambda_{2}}^{2} - \frac{1}{{\lambda_{1}}^{3}} \right)\frac{\partial \psi_{2}}{\partial I_{2}}\\
    P_{22}=2\left(\lambda_{2}-\frac{1}{{\lambda_{1}}^{2}{\lambda_{2}}^{3}}\right)\frac{\partial \psi_{1}}{\partial I_{1}} + 2\left({\lambda_{1}}^{2}\lambda_{2} - \frac{1}{{\lambda_{2}}^{3}}\right)\frac{\partial \psi_{2}}{\partial I_{2}}
\end{align*}
$$

Reference: https://doi.org/10.1016/j.actbio.2024.09.051

In [ ]:
# Definition of stress
def Stress_xx_I5_BT(inputs):
    (dPsidI1, dPsidI2, Stretch, Stretch_z, I1) = inputs
    # stretch_z = 1/(stretch_x*stretch_y)

    # calculate piola stress
    one = tf.constant(1.0,dtype='float32')
    two = tf.constant(2.0,dtype='float32')
    four = tf.constant(4.0,dtype='float32')

    stress_1 = two*dPsidI1*(Stretch-(Stretch_z**two)/Stretch)
    stress_2 = two*dPsidI2 *(one/(Stretch_z**two)/Stretch - Stretch**(-3))

    return stress_1 + stress_2

Finally, we can define seperate stress models for tension/compression, shear and a combination of all loading states.

In [ ]:
# Gradient function
def myGradient(a, b):
    der = tf.gradients(a, b, unconnected_gradients='zero')
    return der[0]

# Complete model architecture definition
def modelArchitecture_I5(Psi_model):
    # Stretch as input
    Stretch_x = keras.layers.Input(shape = (1,),
                                  name = 'Stretch_x')
    Stretch_y = keras.layers.Input(shape = (1,),
                                  name = 'Stretch_y')

    # Specific Invariants BT
    Stretch_z = tf.keras.layers.Lambda(lambda x: 1/(x[0] * x[1]), name = 'lam_z')([Stretch_x, Stretch_y])
    I1_BT = tf.keras.layers.Lambda(lambda x: x[0]**2 + x[1]**2 +x[2]**2, name = 'I1')([Stretch_x, Stretch_y, Stretch_z])
    I2_BT = tf.keras.layers.Lambda(lambda x: (x[0]**2)*(x[1]**2) + 1/x[0]**2 + 1/x[1]**2, name = 'I2')([Stretch_x, Stretch_y])

    # Define Strain Energy
    Psi_BT = Psi_model([I1_BT, I2_BT])

    # Derivative DWdX
    dWI1_BT  = keras.layers.Lambda(lambda x: myGradient(x[0], x[1]))([Psi_BT, I1_BT])
    dWdI2_BT = keras.layers.Lambda(lambda x: myGradient(x[0], x[1]))([Psi_BT, I2_BT])

    # Stress XX
    Stress_xx_BT = keras.layers.Lambda(function = Stress_xx_I5_BT,
                                name = 'Stress_xx_BT')([dWI1_BT, dWdI2_BT,
                                                        Stretch_x, Stretch_z, I1_BT])
    # Stress YY
    Stress_yy_BT = keras.layers.Lambda(function = Stress_xx_I5_BT,
                                name = 'Stress_yy_BT')([dWI1_BT, dWdI2_BT,
                                                        Stretch_y, Stretch_z, I1_BT])

    # Define model
    model_BT = keras.models.Model(inputs=[Stretch_x, Stretch_y],  outputs= [Stress_xx_BT, Stress_yy_BT])

    return model_BT


### 4. Compile model

The compiler definition comprises the loss function definition (here a mean squared error metric), the optimizer (here an Adam optimizer) and the evaluation metric (also mean squared error).

Moreover, we define model callbacks and the keras fit function. The latter obtains the information about which model we want to fit with which data.

In [ ]:
# Optimization utilities
def Compile_and_fit(model_given, input_train, output_train, epochs, path_checkpoint, batch_size, validation_split):

    mse_loss = keras.losses.MeanSquaredError()
    metrics  =[keras.metrics.MeanSquaredError()]
    opti1    = tf.optimizers.Adam(learning_rate=0.001)

    model_given.compile(loss=mse_loss,
                  optimizer=opti1,
                  metrics=metrics)


    es_callback = keras.callbacks.EarlyStopping(monitor="loss", min_delta=0, patience=2000, restore_best_weights=True)

    modelckpt_callback = keras.callbacks.ModelCheckpoint(
    monitor="loss",
    filepath=path_checkpoint,
    verbose=0,
    save_weights_only=True,
    save_best_only=True,
    )

    history = model_given.fit(input_train,
                        output_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        validation_split=validation_split,
                        callbacks=[es_callback, modelckpt_callback],
                        shuffle = True,
                        verbose = 0,
                        # sample_weight = sample_weights
                        )

    return model_given, history

In [ ]:
def R2xy(all_Sigma_xx, all_Sigma_yy, Stress_predicted):
    R2x_all = []
    R2y_all = []
    for k in range(len(all_Sigma_xx)):
        R2x = r2_score_own(all_Sigma_xx[k], Stress_predicted[k][0])
        R2y = r2_score_own(all_Sigma_yy[k], Stress_predicted[k][1])
        R2x_all.append(R2x)
        R2y_all.append(R2y)

    return R2x_all, R2y_all

### 5. Plot functions

Here we define some plot functions to be used to plot the results later on

In [ ]:
ColorI = [1.0, 0.65, 0.0]
ColorS = [0.5, 0.00, 0.0]
#c_lis = ['b','g','r','k','m']
#c_lis = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
c_lis = ['#9c0203', '#80cafd', '#4c7ffe', '#1a34ff', '#070096']

# Useful functions
def r2_score_own(Truth, Prediction):
    R2 = r2_score(Truth,Prediction)
    return max(R2,0.0)

def GetZeroList(model_weights):
    model_zeros = []
    for i in range(len(model_weights)):
        model_zeros.append(np.zeros_like(model_weights[i]))
    return model_zeros

# Loss plot
def plotLoss(axe, history):
    axe.plot(history)
    axe.set_yscale('log')
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')

In [ ]:
# plot the contribution of each term to the model stress prediction
def color_map(ax2, lamx, lamy,lamplot, model_BT, model_weights, Psi_model, cmaplist, terms, label,ticks):
    predictions = np.zeros([lamx.shape[0],terms])
    cmap_r = list(reversed(cmaplist))
    model_plot = copy.deepcopy(model_weights)  # deep copy model weights

    areas=[]
    for i in range(terms):
        model_plot[-1] = np.zeros_like(model_weights[-1])  # wx2 all set to zero
        model_plot[-1][i] = model_weights[-1][i]  # wx2[i] set to trained value

        Psi_model.set_weights(model_plot)
        lower = np.sum(predictions,axis=1)
        if label == 'x':
            upper = lower +  model_BT.predict([lamx, lamy])[0][:].flatten()
            predictions[:,i] = model_BT.predict([lamx, lamy])[0][:].flatten()
            # ax2.fill_between(lamplot[:], lower.flatten(), upper.flatten(), zorder=i+1, lw=0, color=cmap_r[i])
        else:
            upper = lower +  model_BT.predict([lamx, lamy])[1][:].flatten()
            predictions[:,i] = model_BT.predict([lamx, lamy])[1][:].flatten()

        # print(lower,upper)
        p=ax2.fill_between(lamplot[:], lower.flatten(), upper.flatten(), zorder=i+1, lw=0, label=ticks[i], color=cmap_r[i])
        areas.append(p)

    return areas

In [ ]:
# Stress-strain plot with colormap
def plotMap(fig, fig_ax1, Psi_model, model_weights, model_BT, terms, lamx, lamy, lamplot, P_ut_all, Stress_predict_UT, R2,label, cy, titletype):
    tick_arr = list(np.flip(np.arange(terms+4))+0.5)
    tick_label = [r'$I_1$',r'$\exp(I_1)$',r'$I_1^2$',r'$\exp(I_1^2)$',
                  r'$I_2$',r'$\exp(I_2)$',r'$I_2^2$', r'$\exp(I_2^2)$']

    if titletype == 'separate':
      title_label = ["off-x","off-y","equi-biax","strip-x","strip-y"]
    elif  titletype == 'combine':
      title_label = ["off-x","off-y","equi-biax","strip-x/strip-y"]
    end


    cmap = plt.cm.get_cmap('jet',8)   # define the colormap
    cmaplist = [cmap(i) for i in range(cmap.N)]

    myhandles=[]
    s1=fig_ax1.scatter(lamplot, P_ut_all, s=40, zorder=103,lw=1, facecolors='w', edgecolors='k',clip_on=False,label='Data')
    p1=fig_ax1.plot(lamplot, Stress_predict_UT, color='k',zorder=25, lw=1);

    areas=color_map(fig_ax1, lamx, lamy,lamplot, model_BT, model_weights, Psi_model, cmaplist, terms, label,tick_label)


    fig_ax1.text(0.02,0.83,'R2: '+f"{R2:.2f}",transform=fig_ax1.transAxes,fontsize=14, horizontalalignment='left',color='k')

    fig_ax1.set_title(title_label[cy])

    fig_ax1.grid(False)
    fig_ax1.set_xticks([round(min(lamplot),2),round(max(lamplot),2)])
    fig_ax1.set_yticks([round(min(P_ut_all),2),round(max(P_ut_all),2)])


    # plt.tight_layout()
    fig_ax1.set_ylabel(r'Piola stress $P$ [kPa]',fontsize='x-large')
    fig_ax1.set_xlabel(r'stretch $\lambda_'+label+'$ [-]',fontsize='x-large')

    myhandles.append(s1)
    for i in areas:
        myhandles.append(i)
    return myhandles

### 6. Model Training

Parameters and definitions for the model training. Try changing the number of epochs and toggling between the invariant and principal-stretch-based model. Make sure to rename the model_type variable for each test.

In [ ]:
filename_suffix = '_CANN1' #for directory name / version tracking
filename = filename_prefix+filename_suffix

# Training settings
epochs = 10000 #10000
batch_size = 8 #32
validation_split = 0

### Choose regularization type & penalty amount
# Option: 'L1', 'L2'
reg = 'L1'
pen = 0.01  # Use 0 for no regularization 0.001, 0.01, and 0.1
#folder_name = 'L2Reg0.1' # name the folder for your results
#folder_name = str(reg) + 'Reg' + str(pen) # name the folder for your results
folder_name = 'correctedPiola_L1p01'

### Choose which loading modes to train with
modelFit_mode_all = ['all'] # which biaxial mode to use

# Other settings
model_type = 'Model_p'
weight_flag = True
weight_plot_Map_combine = True
weight_plot_Map_seperate = False
train = True

###Set Directories

In [ ]:
# e.g) Make Results/Porcine_skin5/Model_p_0
# The last number increments to prevent overwriting existing folders.
#path2saveResults_0 = path+os.path.join('/Results',filename,model_type+'_'+str(count_directories(os.path.join(os.getcwd(),'Results',filename))))
path2saveResults_0 = path + '/Results/'+filename+'/'+folder_name

makeDIR(path2saveResults_0)
Model_summary = os.path.join(path2saveResults_0, 'Model_summary.txt')
path2saveResults = path2saveResults_0
path2saveResults_check = os.path.join(path2saveResults,'Checkpoints')
makeDIR(path2saveResults)
makeDIR(path2saveResults_check)

In [ ]:
R2_all = np.zeros([10, len(modelFit_mode_all)])
R2_pick = np.zeros([2, len(modelFit_mode_all)])

In [ ]:
#  Training and validation loop
# train = False
for id1, modelFit_mode in enumerate(modelFit_mode_all):
    # Terminal display
    print(40*'=')
    # print('Comp '+str(count)+'/'+str())
    print("Comp {:d} / {:d}".format(id1+1, len(modelFit_mode_all)))
    print(40*'=')
    print("Fitting Mode: ", modelFit_mode)
    print(40*'=')

    # Set directories
    path2saveResults = os.path.join(path2saveResults_0, modelFit_mode)
    path2saveResults_check = os.path.join(path2saveResults,'Checkpoints')
    makeDIR(path2saveResults)
    makeDIR(path2saveResults_check)
    Save_path = path2saveResults + '//model.h5'
    Save_weights = path2saveResults + '//weights'
    Stored_weights = path2saveResults_0 + '//weights'
    path_checkpoint = path2saveResults_check + '//best_weights'

    Psi_model, terms = StrainEnergy_i5(reg,pen)
    model_BT = modelArchitecture_I5(Psi_model)
    model_given = model_BT

    # Model summary
    with open(Model_summary,'w') as fh:
        # Pass the file handle in as a lambda function to make it callable
        Psi_model.summary(line_length=80, print_fn=lambda x: fh.write(x + '\n'))

    # Split training and validation data
    if modelFit_mode == 'all':
        input_train = [np.concatenate(all_lam_x), np.concatenate(all_lam_y)]
        output_train = [np.concatenate(all_Sigma_xx), np.concatenate(all_Sigma_yy)]
    else:
        input_train = [all_lam_x[id1-1], all_lam_y[id1-1]]
        output_train = [all_Sigma_xx[id1-1], all_Sigma_yy[id1-1]]

    if train: # train and save the model
        if modelFit_mode =='all':
            pass
        else:
            model_given.load_weights(os.path.join(path2saveResults_0, 'all') + '//weights', by_name=False, skip_mismatch=False)

        model_given, history = Compile_and_fit(model_given, input_train, output_train,
                                                epochs, path_checkpoint, batch_size, validation_split)

        model_given.load_weights(path_checkpoint, by_name=False, skip_mismatch=False)
        tf.keras.models.save_model(model_given, Save_path, overwrite=True)
        model_given.save_weights(Save_weights, overwrite=True)

        loss_history = history.history['loss']
        fig, axe = plt.subplots(figsize=[6, 5])  # inches
        plotLoss(axe, loss_history)
        plt.savefig(path2saveResults+'/Plot_loss_'+modelFit_mode+'.pdf')
        plt.close()

    else: # load existing model only
        model_given.load_weights(Save_weights, by_name=False, skip_mismatch=False)

    # Get model response
    Stress_predicted = []
    for j in range(len(all_lam_x)):
        Stress_pre = model_BT.predict([all_lam_x[j], all_lam_y[j]])
        Stress_predicted.append(Stress_pre)

    # R2 evaluation
    R2x_all, R2y_all = R2xy(all_Sigma_xx, all_Sigma_yy, Stress_predicted)

    # Terminal Display: End of training
    print('='*30)

    weight_matrix = np.empty((terms, 2))
    for i in range(terms):
      value = Psi_model.get_weights()[i][0][0]
      weight_matrix[i, 0] = value # inner layer is first column
    weight_matrix[:, 1] = Psi_model.get_layer('wx2').get_weights()[0].flatten() # outer layer is second column


    print("weight_matrix")
    print(weight_matrix)


    # Plot Color map
    model_weights_0 = Psi_model.get_weights()
    if weight_plot_Map_seperate:
        for kk in range(len(all_lam_x)):
            fig2 = plt.figure(figsize=(1200/72,600/72))
            spec2 = gridspec.GridSpec(ncols=1, nrows=2, figure=fig2)
            ax1 = fig2.add_subplot(spec2[0,0])
            ax2 = fig2.add_subplot(spec2[1,0])
            plotMap(fig2, ax1, Psi_model, model_weights_0, model_BT,
                            terms, all_lam_x[kk], all_lam_y[kk], all_lam_x[kk], all_Sigma_xx[kk], Stress_predicted[kk][0], R2x_all[kk], 'x',kk,'separate')
            areas = plotMap(fig2, ax2, Psi_model, model_weights_0, model_BT,
                            terms, all_lam_x[kk], all_lam_y[kk], all_lam_y[kk], all_Sigma_yy[kk], Stress_predicted[kk][1], R2x_all[kk], 'y',kk,'separate')

            fig2.tight_layout()
            fig2.legend(handles=areas,loc='outside lower center', ncol=22, fancybox=False)
            plt.savefig(path2saveResults+'//Plot_PI-CANN_MAP_'+modelFit_mode+'_cy_'+str(kk+1)+'.pdf')
            plt.close()

    fig3 = plt.figure(figsize=(1200/72,600/72))
    # figsize=(12.5, 8.33)
    spec3 = gridspec.GridSpec(ncols=4, nrows=2, figure=fig3)
    if weight_plot_Map_combine:
        for kk in range(len(all_lam_x)):
            if kk<4: #x plot: off-x, off-y, equibiax, strip-x
                ax1 = fig3.add_subplot(spec3[0,kk])
                areas=plotMap(fig3, ax1, Psi_model, model_weights_0, model_BT,
                              terms, all_lam_x[kk], all_lam_y[kk], all_lam_x[kk], all_Sigma_xx[kk], Stress_predicted[kk][0], R2x_all[kk],'x',kk,'combine')
            if kk<3: #y plot: off-x, off-y, equibiax
                ax2 = fig3.add_subplot(spec3[1,kk])
                areas=plotMap(fig3, ax2, Psi_model, model_weights_0, model_BT,
                                terms, all_lam_x[kk], all_lam_y[kk], all_lam_y[kk], all_Sigma_yy[kk], Stress_predicted[kk][1], R2y_all[kk],'y',kk,'combine')
            if kk==4: #y plot: strip-y
                ax2 = fig3.add_subplot(spec3[1,kk-1])
                areas=plotMap(fig3, ax2, Psi_model, model_weights_0, model_BT,
                                terms, all_lam_x[kk], all_lam_y[kk], all_lam_y[kk], all_Sigma_yy[kk], Stress_predicted[kk][1], R2y_all[kk],'y',kk-1,'combine')
            if kk==0:
                fig3.legend(handles=areas,loc='outside lower center', ncol=12, frameon=False)
            fig3.tight_layout()
            plt.tight_layout()
        plt.subplots_adjust(bottom=0.2)
        plt.savefig(path2saveResults+'//Plot_PI-CANN_MAP_'+'all'+'.pdf')
        plt.close()

    # Storing data
    Config = {modelFit_mode:modelFit_mode, "R2_x1":R2x_all[0], "R2_x2": R2x_all[1], "R2_x3": R2x_all[2], "R2_x4": R2x_all[3], "R2_x5": R2x_all[4],
            "R2_y1":R2y_all[0], "R2_y2": R2y_all[1], "R2_y3": R2y_all[2], "R2_y4": R2y_all[3], "R2_y5": R2y_all[4],
            "weights": weight_matrix.tolist()}
    json.dump(Config, open(path2saveResults+"//Config_file.txt",'w'))

    weight_matrix_new1 = weight_matrix[:,0]*weight_matrix[:,1]
    weight_matrix_new2 = weight_matrix
    weight_matrix_new2[np.where(weight_matrix_new1 == 0)[0], :] = [0,0]

    numcol = np.arange(23)+1
    func = ['I1','exp(I1)','I1^2','exp(I1^2)',
            'I2','exp(I2)','I2^2', 'exp(I2^2)']
    list_of_tuples = list(zip(numcol,func,weight_matrix_new2))
    df = pd.DataFrame(list_of_tuples , columns=['Count','Function', 'Weight'])
    df.to_csv(path2saveResults+"//Config_file_new.txt", sep='\t', index=False)

    R2_all[:,id1] = np.array(R2x_all + R2y_all)

    if modelFit_mode=='all':
        R2x_p = statistics.mean(R2x_all)
        R2y_p = statistics.mean(R2y_all)
    else:
        R2x_p = R2x_all[id1-1]
        R2y_p = R2y_all[id1-1]

    R2_pick[0,id1] = R2x_p
    R2_pick[1,id1] = R2y_p

    print(40*'=')
    print("Rx: ", R2x_p, "|  Ry:", R2y_p)
    print(40*'=')

    print(df)

In [ ]:
 # Create diagrams
def save_diagram(fig, ax, lamx, lamy, xplot, yplot_1, yplot_2, model_BT, model_weights, Psi_model, terms, label):
    # Save_Diagrams takes in 12 parameters to save NON-LABELED graphs with plots of the experimental data & resulting model

         # ["off-x","off-y","equi-biax","strip-x","strip-y"] -- the different modes


        ax.plot(xplot, yplot_2, color='black', lw=4, zorder=1); # plot the model
        ax.scatter(xplot, yplot_1, edgecolor='black', facecolor='white', linewidth=3,s = 800, alpha=0.9, zorder=200, clip_on = False) # plot the experimental data


       # Running the Color Map Function to plot the colors for each term used in the model

        tick_label = [r'$I_1$',r'$\exp(I_1)$',r'$I_1^2$',r'$\exp(I_1^2)$', # tick_label is needed to run the color_map function
                  r'$I_2$',r'$\exp(I_2)$',r'$I_2^2$',r'$\exp(I_2^2)$']
        cmap = plt.cm.get_cmap('jet',8)
        cmaplist = [cmap(i) for i in range(cmap.N)]
        color_map(ax, lamx, lamy, xplot, model_BT, model_weights, Psi_model, cmaplist, terms, label,tick_label)


      # Extract max and min x/y-values for graph limits
        max_x = np.max(np.array(xplot))
        max_y = np.max(np.array(yplot_1))
        min_x = np.min(np.array(xplot))
        min_y = np.min(np.array(yplot_1))

      # Graph limits and labels
        ax.set_xlim([min_x, max_x])
        ax.set_xticks([min_x, max_x])
        ax.set_xticklabels(['', ''])

        ax.set_ylim([min_y, max_y])
        ax.set_yticks([min_y, max_y])
        ax.set_yticklabels(['', ''])

      # Creates a new path called "Diagrams" to save all graphs
        newpath = path2saveResults + '/Diagrams/'
        if not os.path.exists(newpath):
           os.makedirs(newpath)

      # Saves figure
        plt.savefig(newpath + title_name  + '.png', pad_inches = 2) # creates a padding white border so the dot plots dont get cut off
end

# Each section creates a different figure. Manually select the lamx, lamy, xploy, yploy_1, yplot_2 within pre-existing variables. Calls save_diagram function to create untitled figures

title_name =  'lx_off-x'   # "lambda x, off-x diretion, ""
fig1, ax1 = plt.subplots(figsize=(12.5, 8.33))
lamx = all_lam_x[0]
lamy = all_lam_y[0]
xplot = all_lam_x[0]
yplot_1 = all_Sigma_xx[0]
yplot_2 = Stress_predicted[0][0]
save_diagram(fig1, ax1, lamx, lamy, xplot, yplot_1, yplot_2, model_BT, model_weights_0, Psi_model, terms, 'x')

title_name =  'lx_off-y'   # "lambda x  " off-y diretion, ""
fig2, ax2 = plt.subplots(figsize=(12.5, 8.33))
lamx = all_lam_x[1]
lamy = all_lam_y[1]
xplot = all_lam_x[1]
yplot_1 = all_Sigma_xx[1]
yplot_2 = Stress_predicted[1][0]
save_diagram(fig2, ax2, lamx, lamy, xplot, yplot_1, yplot_2, model_BT, model_weights_0, Psi_model, terms, 'x')

title_name =  'lx_equibiax'   # " lambda x equi_biax diretion ""
fig3, ax3 = plt.subplots(figsize=(12.5, 8.33))
lamx = all_lam_x[2]
lamy = all_lam_y[2]
xplot = all_lam_x[2]
yplot_1 = all_Sigma_xx[2]
yplot_2 = Stress_predicted[2][0]
save_diagram(fig3, ax3, lamx, lamy, xplot, yplot_1, yplot_2, model_BT, model_weights_0, Psi_model, terms, 'x')

title_name =  'lx_strip-xstrip-y'   #  lambd x, strip-xstripy diretion""
fig4, ax4 = plt.subplots(figsize=(12.5, 8.33))
lamx = all_lam_x[3]
lamy = all_lam_y[3]
xplot = all_lam_x[3]
yplot_1 = all_Sigma_xx[3]
yplot_2 = Stress_predicted[3][0]
save_diagram(fig4, ax4, lamx, lamy, xplot, yplot_1, yplot_2, model_BT, model_weights_0, Psi_model, terms, 'x')

title_name =  'ly_off-x'   # "lambda y , off-x diretion,  ""
fig5, ax5 = plt.subplots(figsize=(12.5, 8.33))
lamx = all_lam_x[0]
lamy = all_lam_y[0]
xplot = all_lam_y[0]
yplot_1 = all_Sigma_yy[0]
yplot_2 = Stress_predicted[0][1]
save_diagram(fig5, ax5, lamx, lamy, xplot, yplot_1, yplot_2, model_BT, model_weights_0, Psi_model, terms, 'y')

title_name =  'ly_off-y'   # "lambda y, off-y diretion""
fig6, ax6 = plt.subplots(figsize=(12.5, 8.33))
lamx = all_lam_x[1]
lamy = all_lam_y[1]
xplot = all_lam_y[1]
yplot_1 = all_Sigma_yy[1]
yplot_2 = Stress_predicted[1][1]
save_diagram(fig6, ax6, lamx, lamy, xplot, yplot_1, yplot_2, model_BT, model_weights_0, Psi_model, terms, 'y')

title_name =  'ly_equibiax'   # "lambda y, equi_biax diretion ""
fig7, ax7 = plt.subplots(figsize=(12.5, 8.33))
lamx = all_lam_x[2]
lamy = all_lam_y[2]
xplot = all_lam_y[2]
yplot_1 = all_Sigma_yy[2]
yplot_2 = Stress_predicted[2][1]
save_diagram(fig7, ax7, lamx, lamy, xplot, yplot_1, yplot_2, model_BT, model_weights_0, Psi_model, terms, 'y')

title_name =  'ly_strip-xstrip-y'    # lambda y, strip-xstripy diretion ""
fig8, ax8 = plt.subplots(figsize=(12.5, 8.33))
lamx = all_lam_x[4]
lamy = all_lam_y[4]
xplot = all_lam_y[4]
yplot_1 = all_Sigma_yy[4]
yplot_2 = Stress_predicted[4][1]
save_diagram(fig8, ax8, lamx, lamy, xplot, yplot_1, yplot_2, model_BT, model_weights_0, Psi_model, terms, 'y')

